In [2]:
!pip install --user -U bert-serving-server bert-serving-client

  Using cached https://files.pythonhosted.org/packages/af/ef/0df9a6ce54a02d0a891d25af60e49b3a3a64d425e80c28acfee97f5ab0f2/bert_serving_server-1.9.6-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/b5/2f/dd50af5b8dbde79e69f4bd2edec222eaa23d1015b03e9613411b78f9a639/bert_serving_client-1.9.6-py2.py3-none-any.whl


In [3]:
!pip install --user tensorflow

In [4]:
# REf: https://modelzoo.co/model/pytorch-pretrained-bert

!pip install --user pytorch-pretrained-bert

     |████████████████████████████████| 133kB 3.4MB/s eta 0:00:01
     |████████████████████████████████| 655kB 19.2MB/s eta 0:00:01
  Created wheel for regex: filename=regex-2019.6.8-cp36-cp36m-linux_x86_64.whl size=604148 sha256=9c07c950133add80ea6a60f45ae7834e1736da275f6beca37b02684b8b5eebd4
  Stored in directory: /home/developer/.cache/pip/wheels/35/e4/80/abf3b33ba89cf65cd262af8a22a5a999cc28fbfabea6b38473
Successfully built regex


In [5]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenized input
text = "Who was Jim Henson ? Jim Henson was a puppeteer"
tokenized_text = tokenizer.tokenize(text)

# Mask a token that we will try to predict back with `BertForMaskedLM`
masked_index = 6
tokenized_text[masked_index] = '[MASK]'
assert tokenized_text == ['who', 'was', 'jim', 'henson', '?', 'jim', '[MASK]', 'was', 'a', 'puppet', '##eer']

# Convert token to vocabulary indices
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
# Define sentence A and B indices associated to 1st and 2nd sentences (see paper)
segments_ids = [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]

# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

100%|██████████| 231508/231508 [00:00<00:00, 6320559.60B/s]


In [6]:
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased')
model.eval()

# Predict hidden states features for each layer
encoded_layers, _ = model(tokens_tensor, segments_tensors)
# We have a hidden states for each of the 12 layers in model bert-base-uncased
assert len(encoded_layers) == 12

100%|██████████| 407873900/407873900 [00:33<00:00, 12257748.07B/s]


In [ ]:
# Load pre-trained model (weights)
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.eval()

# Predict all tokens
predictions = model(tokens_tensor, segments_tensors)

# confirm we were able to predict 'henson'
predicted_index = torch.argmax(predictions[0, masked_index]).item()
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])
assert predicted_token == 'henson'